In [1]:
import os
import numpy as np
import tensorflow as tf
from vgg_network import VGG_Network
from data_utils import train_input_fn, test_input_fn, get_csv_dataset
from preprocessing_cnn import run_images_in_cnn

class_nbr = 6
target_w = 224
input_shape = [None, target_w, target_w, 3]
batch_size = 2

cnn_outputs_dir = os.path.join('..', 'tmp', 'cnn_output')
if not os.path.exists(cnn_outputs_dir):
    os.makedirs(cnn_outputs_dir)
    
tf.logging.set_verbosity(tf.logging.DEBUG)

/Users/Sylvain/.pyenv/versions/3.6.4/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# VGG16

In [2]:
# Network
input_plh = tf.placeholder(tf.float32, shape=input_shape, name="X")
vgg_model = VGG_Network(include_FC_head=False)
vgg_network = vgg_model(input_plh)
network_outputs = vgg_network.outputs

# Pretrained weights loader hook
hook_load_pretrained = lambda sess: vgg_model.load_pretrained(sess)

[TL] InputLayer  vgg_network/input: (?, 224, 224, 3)
[TL] Conv2dLayer vgg_network/conv_layers/h1/conv1: shape:(3, 3, 3, 64) strides:(1, 1, 1, 1) pad:SAME act:relu
[TL] Conv2dLayer vgg_network/conv_layers/h1/conv2: shape:(3, 3, 64, 64) strides:(1, 1, 1, 1) pad:SAME act:relu
[TL] PoolLayer   vgg_network/conv_layers/h1/pool: ksize:[1, 2, 2, 1] strides:[1, 2, 2, 1] padding:SAME pool:max_pool
[TL] Conv2dLayer vgg_network/conv_layers/h2/conv1: shape:(3, 3, 64, 128) strides:(1, 1, 1, 1) pad:SAME act:relu
[TL] Conv2dLayer vgg_network/conv_layers/h2/conv2: shape:(3, 3, 128, 128) strides:(1, 1, 1, 1) pad:SAME act:relu
[TL] PoolLayer   vgg_network/conv_layers/h2/pool: ksize:[1, 2, 2, 1] strides:[1, 2, 2, 1] padding:SAME pool:max_pool
[TL] Conv2dLayer vgg_network/conv_layers/h3/conv1: shape:(3, 3, 128, 256) strides:(1, 1, 1, 1) pad:SAME act:relu
[TL] Conv2dLayer vgg_network/conv_layers/h3/conv2: shape:(3, 3, 256, 256) strides:(1, 1, 1, 1) pad:SAME act:relu
[TL] Conv2dLayer vgg_network/conv_layers/

### Train set

In [3]:
# Train Dataset
train_csv_path = os.path.join("..", "data/DAGM 2007 - Splitted", str(class_nbr), "{}_files.csv".format("train"))
input_fn_images = train_input_fn(class_nbr, target_w)
input_fn_filenames = get_csv_dataset(train_csv_path, class_nbr).map(lambda fn, label: fn)
input_fn = tf.data.Dataset.zip((input_fn_filenames, input_fn_images)).batch(batch_size)
iterator_op = input_fn.make_one_shot_iterator().get_next()

DEBUG:tensorflow: Created a train dataset from csv file ../data/DAGM 2007 - Splitted/6/train_files.csv
DEBUG:tensorflow:     Output shapes : (224, 224, 3)
DEBUG:tensorflow:     Output types : <dtype: 'float32'>



In [4]:
# Target directory 
vgg16_train_output_dir = os.path.join(cnn_outputs_dir, '{}/{}/train'.format('VGG16', class_nbr))
os.makedirs(vgg16_train_output_dir) if not os.path.exists(vgg16_train_output_dir) else None

run_images_in_cnn(
    iterator_op, 
    input_plh, 
    network_outputs, 
    vgg16_train_output_dir, 
    hooks=[hook_load_pretrained]
)

INFO:tensorflow:Running images through the network and saving results in ../tmp/cnn_output/VGG16/6/train
INFO:tensorflow:Results will be saved to '../tmp/cnn_output/VGG16/6/train'
INFO:tensorflow:Loading VGG Net weights ...
INFO:tensorflow:Finished loading VGG Net weights ...


KeyboardInterrupt: 

## Test set

In [ ]:
# Test Dataset
test_csv_path = os.path.join("..", "data/DAGM 2007 - Splitted", str(class_nbr), "{}_files.csv".format("train"))
input_fn_images = test_input_fn(class_nbr, target_w).map(lambda img, label: img)
input_fn_filenames = get_csv_dataset(test_csv_path, class_nbr).map(lambda fn, label: fn)
input_fn = tf.data.Dataset.zip((input_fn_filenames, input_fn_images)).batch(batch_size)
iterator_op = input_fn.make_one_shot_iterator().get_next()

In [ ]:
# Target directory 
vgg16_train_output_dir = os.path.join(cnn_outputs_dir, '{}/{}/test'.format('VGG16', class_nbr))
os.makedirs(vgg16_train_output_dir) if not os.path.exists(vgg16_train_output_dir) else None

run_images_in_cnn(
    iterator_op, 
    input_plh, 
    network_outputs, 
    vgg16_train_output_dir, 
    hooks=[hook_load_pretrained]
)